# Environment Setup

This notebook captures system information for benchmarking and saves it to `environment.json`.


Figures are saved to `../figures/` and tables to `../results/`.


In [ ]:
from pathlib import Path
FIGURES_DIR = Path('../figures')
RESULTS_DIR = Path('../results')
FIGURES_DIR.mkdir(exist_ok=True)
RESULTS_DIR.mkdir(exist_ok=True)
try:
    import ipynbname
    NB_NAME = ipynbname.path().stem
except Exception:  # pragma: no cover
    NB_NAME = 'notebook'


In [ ]:
import json, platform, subprocess, psutil, numpy as np, os
from pathlib import Path


def get_cpu_model():
    system = platform.system()
    if system == "Linux":
        try:
            with open("/proc/cpuinfo") as f:
                for line in f:
                    if "model name" in line:
                        return line.split(":", 1)[1].strip()
        except Exception:
            pass
    elif system == "Darwin":
        try:
            return subprocess.check_output(
                ["sysctl", "-n", "machdep.cpu.brand_string"], text=True
            ).strip()
        except Exception:
            pass
    return platform.processor()

info = {}
info["cpu_model"] = get_cpu_model()
info["cpu_count"] = psutil.cpu_count()
info["ram_gb"] = round(psutil.virtual_memory().total / 1024**3, 2)
info["os"] = platform.platform()
info["python_version"] = platform.python_version()
info["blas_libraries"] = getattr(np.__config__, "blas_opt_info", {}).get("libraries", [])
info["lapack_libraries"] = getattr(np.__config__, "lapack_opt_info", {}).get("libraries", [])
import stim, mqt.ddsim, mqt.core
info["stim_version"] = getattr(stim, "__version__", None)
info["mqt_dd_version"] = getattr(mqt.ddsim, "__version__", None)
info["mqt_core_version"] = getattr(mqt.core, "__version__", None)
try:
    import pybind11
    info["pybind11_version"] = pybind11.__version__
except Exception:
    info["pybind11_version"] = None
info["quasar_commit"] = subprocess.check_output(["git","rev-parse","HEAD"]).decode().strip()

def run_cmd(cmd):
    try:
        return subprocess.check_output(cmd, text=True).splitlines()[0]
    except Exception:
        return None
info["compiler"] = run_cmd(["cc","--version"]) or run_cmd(["gcc","--version"])
info["cmake_version"] = run_cmd(["cmake","--version"])
info["QUASAR_QUICK_MAX_QUBITS"] = os.getenv("QUASAR_QUICK_MAX_QUBITS")
info["QUASAR_QUICK_MAX_GATES"] = os.getenv("QUASAR_QUICK_MAX_GATES")
info["QUASAR_QUICK_MAX_DEPTH"] = os.getenv("QUASAR_QUICK_MAX_DEPTH")
info["QUASAR_MPS_TARGET_FIDELITY"] = os.getenv("QUASAR_MPS_TARGET_FIDELITY")
Path("environment.json").write_text(json.dumps(info, indent=2, default=str))
info

Environment information saved to `environment.json`.


## Benchmark Circuits

The table below lists benchmark circuits used in QuASAr.

In [2]:
from benchmarks.circuits import (
    clifford_ec_circuit,
    ripple_add_circuit,
    vqe_chain_circuit,
    random_hybrid_circuit,
    recur_subroutine_circuit,
)
import inspect
from pathlib import Path
from IPython.display import Markdown


def src_link(func):
    file = Path(inspect.getsourcefile(func)).relative_to(Path.cwd())
    line = inspect.getsourcelines(func)[1]
    return f"[generator]({file}#L{line})"

circuits = [
    ("Clifford-EC", clifford_ec_circuit, "3-qubit bit-flip error correction"),
    ("Ripple-Add", ripple_add_circuit, "Ripple-carry adder for two 4-bit registers"),
    ("VQE-Chain", vqe_chain_circuit, "VQE ansatz with linear entanglement chain"),
    ("Random-Hybrid", random_hybrid_circuit, "Random mix of Clifford and T gates"),
    ("Recur-Subroutine", recur_subroutine_circuit, "Circuit with repeated subroutine layers"),
]

rows = ["| Circuit | Qubits | Gates | Description | Source |", "|---|---|---|---|---|"]
for name, fn, desc in circuits:
    circ = fn()
    rows.append(f"| {name} | {circ.num_qubits} | {circ.num_gates} | {desc} | {src_link(fn)} |")
Markdown("\n".join(rows))


| Circuit | Qubits | Gates | Description | Source |
|---|---|---|---|---|
| Clifford-EC | 5 | 9 | 3-qubit bit-flip error correction | [generator](benchmarks/circuits.py#L419) |
| Ripple-Add | 13 | 203 | Ripple-carry adder for two 4-bit registers | [generator](benchmarks/circuits.py#L437) |
| VQE-Chain | 6 | 46 | VQE ansatz with linear entanglement chain | [generator](benchmarks/circuits.py#L446) |
| Random-Hybrid | 6 | 33 | Random mix of Clifford and T gates | [generator](benchmarks/circuits.py#L454) |
| Recur-Subroutine | 4 | 18 | Circuit with repeated subroutine layers | [generator](benchmarks/circuits.py#L472) |

In [3]:

# Record parameters and results
import json, pathlib
try:
    import ipynbname
    nb_name = ipynbname.path().stem
except Exception:  # pragma: no cover
    nb_name = 'notebook'

# Collect simple parameters from globals
_params = {
    k: v for k, v in globals().items()
    if not k.startswith('_') and isinstance(v, (int, float, str, bool, list, dict, tuple))
}
pathlib.Path('../results').mkdir(exist_ok=True)
with open(f"../results/{nb_name}_params.json", 'w') as f:
    json.dump(_params, f, indent=2, default=str)
if 'results' in globals():
    try:
        with open(f"../results/{nb_name}_results.json", 'w') as f:
            json.dump(results, f, indent=2, default=str)
    except TypeError:
        pass
print(json.dumps(_params, indent=2, default=str))

{
  "In": [
    "",
    "import json, platform, subprocess, psutil, numpy as np, os\nfrom pathlib import Path\n\ninfo = {}\ninfo['cpu_model'] = next((l.split(':',1)[1].strip() for l in open('/proc/cpuinfo') if 'model name' in l), platform.processor())\ninfo['cpu_count'] = psutil.cpu_count()\ninfo['ram_gb'] = round(psutil.virtual_memory().total / 1024**3, 2)\ninfo['os'] = platform.platform()\ninfo['python_version'] = platform.python_version()\ninfo['blas_libraries'] = getattr(np.__config__, 'blas_opt_info', {}).get('libraries', [])\ninfo['lapack_libraries'] = getattr(np.__config__, 'lapack_opt_info', {}).get('libraries', [])\nimport stim, mqt.ddsim, mqt.core\ninfo['stim_version'] = getattr(stim, '__version__', None)\ninfo['mqt_dd_version'] = getattr(mqt.ddsim, '__version__', None)\ninfo['mqt_core_version'] = getattr(mqt.core, '__version__', None)\ntry:\n    import pybind11\n    info['pybind11_version'] = pybind11.__version__\nexcept Exception:\n    info['pybind11_version'] = None\ninfo[

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
for i, num in enumerate(plt.get_fignums(), start=1):
    plt.figure(num)
    plt.savefig(FIGURES_DIR / f'{NB_NAME}_plot{i}.png')
for _name, _obj in globals().items():
    if isinstance(_obj, pd.DataFrame):
        _obj.to_csv(RESULTS_DIR / f'{NB_NAME}_{_name}.csv', index=False)
        try:
            _obj.to_latex(RESULTS_DIR / f'{NB_NAME}_{_name}.tex', index=False)
        except Exception:
            pass
